<a href="https://colab.research.google.com/github/Rameshkumar789/Computational-Menthods-INFO-5731/blob/main/Korlakunta_In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [1]:
# Write your code here
import numpy as np
import pandas as pd
import re 
import nltk 
from google.colab import files
from nltk.corpus import stopwords

In [2]:
uploaded = files.upload()

Saving stsa-test.txt to stsa-test.txt
Saving stsa-train.txt to stsa-train.txt


In [3]:
raw_data = open('/content/stsa-train.txt').read()

In [4]:
test_data=open('/content/stsa-test.txt').read()

In [5]:
def prepare_data(data):
    i=[]
    j=[]
    for line in data.split("\n"):
        text = line.split()
        i.append(text[0])
        j.append(" ".join(text[1:]))
    return j, i 


## Train Data

In [6]:
content, sentiment = prepare_data(raw_data)
temp = list(zip(content, sentiment))
data_train = pd.DataFrame(temp,columns=['Content', 'Sentiment'])

In [7]:
data_train

,Content,Sentiment
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
...,...,...
6915,"painful , horrifying and oppressively tragic ,...",1
6916,take care is nicely performed by a quintet of ...,0
6917,"the script covers huge , heavy topics in a bla...",0
6918,a seriously bad film with seriously warped log...,0


## Test Data

In [8]:
content, sentiment = prepare_data(test_data)
temp = list(zip(content, sentiment))
data_test = pd.DataFrame(temp,columns=['Content', 'Sentiment'])

In [9]:
data_test

,Content,Sentiment
0,"no movement , no yuks , not much of anything .",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films .,1
...,...,...
1816,"an often-deadly boring , strange reading of a ...",0
1817,the problem with concept films is that if the ...,0
1818,"safe conduct , however ambitious and well-inte...",0
1819,"a film made with as little wit , interest , an...",0


## Train Data Cleaning

In [10]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
data_train['Stop_Content'] = data_train['Content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
data_train = data_train[['Content', 'Stop_Content', 'Sentiment']]

In [12]:
data_train

,Content,Stop_Content,Sentiment
0,"a stirring , funny and finally transporting re...","stirring , funny finally transporting re-imagi...",1
1,apparently reassembled from the cutting-room f...,apparently reassembled cutting-room floor give...,0
2,they presume their audience wo n't sit still f...,presume audience wo n't sit still sociology le...,0
3,this is a visually stunning rumination on love...,"visually stunning rumination love , memory , h...",1
4,jonathan parker 's bartleby should have been t...,jonathan parker 's bartleby be-all-end-all mod...,1
...,...,...,...
6915,"painful , horrifying and oppressively tragic ,...","painful , horrifying oppressively tragic , fil...",1
6916,take care is nicely performed by a quintet of ...,"take care nicely performed quintet actresses ,...",0
6917,"the script covers huge , heavy topics in a bla...","script covers huge , heavy topics bland , surf...",0
6918,a seriously bad film with seriously warped log...,seriously bad film seriously warped logic writ...,0


## Test Data Cleaning

In [13]:
data_test['Stop_Content'] = data_test['Content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [14]:
data_test = data_test[['Content', 'Stop_Content', 'Sentiment']]

In [15]:
data_test

,Content,Stop_Content,Sentiment
0,"no movement , no yuks , not much of anything .","movement , yuks , much anything .",0
1,"a gob of drivel so sickly sweet , even the eag...","gob drivel sickly sweet , even eager consumers...",0
2,"gangs of new york is an unapologetic mess , wh...","gangs new york unapologetic mess , whose savin...",0
3,"we never really feel involved with the story ,...","never really feel involved story , ideas remai...",0
4,this is one of polanski 's best films .,one polanski 's best films .,1
...,...,...,...
1816,"an often-deadly boring , strange reading of a ...","often-deadly boring , strange reading classic ...",0
1817,the problem with concept films is that if the ...,"problem concept films concept poor one , 's sa...",0
1818,"safe conduct , however ambitious and well-inte...","safe conduct , however ambitious well-intentio...",0
1819,"a film made with as little wit , interest , an...","film made little wit , interest , professional...",0


## Model

In [16]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing


from sklearn.model_selection import cross_validate

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [17]:
kfold = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=1)

In [18]:
# Feature Selection
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=2000)
tfidf_vect.fit(data_train['Stop_Content'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)

In [19]:
def transform_train_data(train, test, vect):
    x_train =  vect.transform(train)
    x_test =  tfidf_vect.transform(test)
    
    return x_train, x_test

In [20]:
def transform_test_data(train, test):
  encoder = preprocessing.LabelEncoder()
  train_y = encoder.fit_transform(train)
  valid_y = encoder.fit_transform(test)
  
  return train_y, valid_y

### Test Data

In [21]:
test_data_x =  tfidf_vect.transform(data_test['Stop_Content'])
encoder = preprocessing.LabelEncoder()
test_data_y = encoder.fit_transform(data_test['Sentiment'])

### MutinomialNB

In [22]:
pred_accuracy_score=[]
pred_precison_score=[]
pred_recall_score=[]
pred_f1_score=[]

In [23]:
clf = MultinomialNB()

In [24]:
for train_x, test_x in kfold.split(data_train['Stop_Content'], data_train['Sentiment']):
  train_X, test_X = data_train['Stop_Content'][train_x], data_train['Stop_Content'][test_x]
  train_y, test_y = data_train['Sentiment'][train_x], data_train['Sentiment'][test_x]
  temp_train_x, temp_test_x = transform_train_data(train_X, test_X, tfidf_vect)
  temp_train_y, temp_test_y = transform_test_data(train_y, test_y)

  clf.fit(temp_train_x, temp_train_y)
  pred_y = clf.predict(temp_test_x)

  pred_accuracy_score.append(accuracy_score(temp_test_y, pred_y))
  pred_precison_score.append(precision_score(temp_test_y, pred_y))
  pred_recall_score.append(recall_score(temp_test_y, pred_y))
  pred_f1_score.append(f1_score(temp_test_y, pred_y))

In [25]:
print("Train Data\n")
print("Accuracy Score: ", pred_accuracy_score)
print("Precision Score: ", pred_precison_score)
print("Recall Score: ", pred_recall_score)
print("F1 Score: ", pred_f1_score)

Train Data

Accuracy Score:  [0.7673410404624278, 0.773121387283237, 0.7666184971098265, 0.763728323699422, 0.7579479768786127, 0.7832369942196532, 0.7803468208092486, 0.759393063583815, 0.7658959537572254, 0.7738439306358381]
Precision Score:  [0.7583979328165374, 0.765625, 0.7642384105960265, 0.7581699346405228, 0.763265306122449, 0.7747395833333334, 0.7764550264550265, 0.7542483660130719, 0.7632275132275133, 0.7680209698558322]
Recall Score:  [0.8130193905817175, 0.814404432132964, 0.7991689750692521, 0.8033240997229917, 0.7770083102493075, 0.8240997229916898, 0.8130193905817175, 0.7991689750692521, 0.7991689750692521, 0.8116343490304709]
F1 Score:  [0.7847593582887701, 0.789261744966443, 0.7813134732566013, 0.7800941492938803, 0.7700754975978037, 0.7986577181208054, 0.7943166441136672, 0.7760591795561533, 0.7807848443843032, 0.7892255892255893]


In [26]:
pred_y=clf.predict(test_data_x)
print("Test Data\n")
print("Accuracy Score: ", accuracy_score(test_data_y, pred_y))
print("Precision Score: ", precision_score(test_data_y, pred_y))
print("Recall Score: ", recall_score(test_data_y, pred_y))
print("F1 Score: ", f1_score(test_data_y, pred_y))


Test Data

Accuracy Score:  0.7693574958813838
Precision Score:  0.7452357071213641
Recall Score:  0.8173817381738174
F1 Score:  0.7796432318992655


### SVM

In [27]:
pred_accuracy_score=[]
pred_precison_score=[]
pred_recall_score=[]
pred_f1_score=[]

In [28]:
clf = LinearSVC()

In [29]:
for train_x, test_x in kfold.split(data_train['Stop_Content'], data_train['Sentiment']):
  train_X, test_X = data_train['Stop_Content'][train_x], data_train['Stop_Content'][test_x]
  train_y, test_y = data_train['Sentiment'][train_x], data_train['Sentiment'][test_x]
  temp_train_x, temp_test_x = transform_train_data(train_X, test_X, tfidf_vect)
  temp_train_y, temp_test_y = transform_test_data(train_y, test_y)

  clf.fit(temp_train_x, temp_train_y)
  pred_y = clf.predict(temp_test_x)

  pred_accuracy_score.append(accuracy_score(temp_test_y, pred_y))
  pred_precison_score.append(precision_score(temp_test_y, pred_y))
  pred_recall_score.append(recall_score(temp_test_y, pred_y))
  pred_f1_score.append(f1_score(temp_test_y, pred_y))

In [30]:
print("Train Data\n")
print("Accuracy Score: ", pred_accuracy_score)
print("Precision Score: ", pred_precison_score)
print("Recall Score: ", pred_recall_score)
print("F1 Score: ", pred_f1_score)

Train Data

Accuracy Score:  [0.7536127167630058, 0.7658959537572254, 0.7528901734104047, 0.7586705202312138, 0.7521676300578035, 0.7752890173410405, 0.7586705202312138, 0.7507225433526011, 0.7471098265895953, 0.755057803468208]
Precision Score:  [0.7584803256445047, 0.7674731182795699, 0.7668539325842697, 0.7701949860724234, 0.7695590327169275, 0.7788331071913162, 0.7724719101123596, 0.757162346521146, 0.7534059945504087, 0.7678321678321678]
Recall Score:  [0.7742382271468145, 0.7908587257617729, 0.7562326869806094, 0.7659279778393352, 0.7493074792243767, 0.7950138504155124, 0.7617728531855956, 0.7686980609418282, 0.7659279778393352, 0.760387811634349]
F1 Score:  [0.7662782727895819, 0.7789904502046384, 0.7615062761506276, 0.7680555555555556, 0.7592982456140351, 0.7868403015764222, 0.7670850767085077, 0.7628865979381444, 0.7596153846153847, 0.7640918580375783]


In [31]:
pred_y=clf.predict(test_data_x)
print("Test Data\n")
print("Accuracy Score: ", accuracy_score(test_data_y, pred_y))
print("Precision Score: ", precision_score(test_data_y, pred_y))
print("Recall Score: ", recall_score(test_data_y, pred_y))
print("F1 Score: ", f1_score(test_data_y, pred_y))

Test Data

Accuracy Score:  0.7473915431081823
Precision Score:  0.7340980187695516
Recall Score:  0.7744774477447744
F1 Score:  0.7537473233404711


### KNN

In [32]:
pred_accuracy_score=[]
pred_precison_score=[]
pred_recall_score=[]
pred_f1_score=[]

In [33]:
clf = KNeighborsClassifier()

In [34]:
for train_x, test_x in kfold.split(data_train['Stop_Content'], data_train['Sentiment']):
  train_X, test_X = data_train['Stop_Content'][train_x], data_train['Stop_Content'][test_x]
  train_y, test_y = data_train['Sentiment'][train_x], data_train['Sentiment'][test_x]
  temp_train_x, temp_test_x = transform_train_data(train_X, test_X, tfidf_vect)
  temp_train_y, temp_test_y = transform_test_data(train_y, test_y)

  clf.fit(temp_train_x, temp_train_y)
  pred_y = clf.predict(temp_test_x)

  pred_accuracy_score.append(accuracy_score(temp_test_y, pred_y))
  pred_precison_score.append(precision_score(temp_test_y, pred_y))
  pred_recall_score.append(recall_score(temp_test_y, pred_y))
  pred_f1_score.append(f1_score(temp_test_y, pred_y))

In [35]:
print("Train Data\n")
print("Accuracy Score: ", pred_accuracy_score)
print("Precision Score: ", pred_precison_score)
print("Recall Score: ", pred_recall_score)
print("F1 Score: ", pred_f1_score)

Train Data

Accuracy Score:  [0.5332369942196532, 0.5354046242774566, 0.5325144508670521, 0.5115606936416185, 0.5245664739884393, 0.5057803468208093, 0.5289017341040463, 0.5339595375722543, 0.5173410404624278, 0.5303468208092486]
Precision Score:  [0.589622641509434, 0.5824634655532359, 0.5886524822695035, 0.5631868131868132, 0.5796019900497512, 0.5465686274509803, 0.57847533632287, 0.6005221932114883, 0.5560165975103735, 0.5746887966804979]
Recall Score:  [0.3462603878116344, 0.3864265927977839, 0.3448753462603878, 0.2839335180055402, 0.32271468144044324, 0.30886426592797783, 0.3573407202216066, 0.3185595567867036, 0.37119113573407203, 0.38365650969529086]
F1 Score:  [0.43630017452006986, 0.4646128226477935, 0.4349344978165938, 0.37753222836095757, 0.41459074733096085, 0.39469026548672564, 0.4417808219178082, 0.416289592760181, 0.4451827242524917, 0.46013289036544847]


In [36]:
pred_y=clf.predict(test_data_x)
print("Test Data\n")
print("Accuracy Score: ", accuracy_score(test_data_y, pred_y))
print("Precision Score: ", precision_score(test_data_y, pred_y))
print("Recall Score: ", recall_score(test_data_y, pred_y))
print("F1 Score: ", f1_score(test_data_y, pred_y))

Test Data

Accuracy Score:  0.5370675453047776
Precision Score:  0.5557432432432432
Recall Score:  0.3619361936193619
F1 Score:  0.43837441705529645


### Decision Tree

In [37]:
pred_accuracy_score=[]
pred_precison_score=[]
pred_recall_score=[]
pred_f1_score=[]

In [38]:
clf = DecisionTreeClassifier()

In [39]:
for train_x, test_x in kfold.split(data_train['Stop_Content'], data_train['Sentiment']):
  train_X, test_X = data_train['Stop_Content'][train_x], data_train['Stop_Content'][test_x]
  train_y, test_y = data_train['Sentiment'][train_x], data_train['Sentiment'][test_x]
  temp_train_x, temp_test_x = transform_train_data(train_X, test_X, tfidf_vect)
  temp_train_y, temp_test_y = transform_test_data(train_y, test_y)

  clf.fit(temp_train_x, temp_train_y)
  pred_y = clf.predict(temp_test_x)

  pred_accuracy_score.append(accuracy_score(temp_test_y, pred_y))
  pred_precison_score.append(precision_score(temp_test_y, pred_y))
  pred_recall_score.append(recall_score(temp_test_y, pred_y))
  pred_f1_score.append(f1_score(temp_test_y, pred_y))

In [40]:
print("Train Data\n")
print("Accuracy Score: ", pred_accuracy_score)
print("Precision Score: ", pred_precison_score)
print("Recall Score: ", pred_recall_score)
print("F1 Score: ", pred_f1_score)

Train Data

Accuracy Score:  [0.6502890173410405, 0.6452312138728323, 0.6539017341040463, 0.6401734104046243, 0.671242774566474, 0.6690751445086706, 0.6575144508670521, 0.6510115606936416, 0.6604046242774566, 0.6473988439306358]
Precision Score:  [0.6755162241887905, 0.6638297872340425, 0.6907378335949764, 0.6613832853025937, 0.698956780923994, 0.7018348623853211, 0.6890243902439024, 0.6780923994038748, 0.6826086956521739, 0.6720588235294118]
Recall Score:  [0.6343490304709142, 0.6481994459833795, 0.6094182825484764, 0.6357340720221607, 0.649584487534626, 0.6357340720221607, 0.6260387811634349, 0.6301939058171745, 0.6523545706371191, 0.6329639889196675]
F1 Score:  [0.6542857142857144, 0.6559215136650315, 0.6475349521707137, 0.6483050847457628, 0.6733668341708542, 0.6671511627906977, 0.6560232220609579, 0.6532663316582915, 0.6671388101983002, 0.6519258202567761]


In [41]:
pred_y=clf.predict(test_data_x)
print("Test Data\n")
print("Accuracy Score: ", accuracy_score(test_data_y, pred_y))
print("Precision Score: ", precision_score(test_data_y, pred_y))
print("Recall Score: ", recall_score(test_data_y, pred_y))
print("F1 Score: ", f1_score(test_data_y, pred_y))

Test Data

Accuracy Score:  0.6523887973640856
Precision Score:  0.6564625850340136
Recall Score:  0.636963696369637
F1 Score:  0.6465661641541038


### Random Forest

In [42]:
pred_accuracy_score=[]
pred_precison_score=[]
pred_recall_score=[]
pred_f1_score=[]

In [43]:
clf = RandomForestClassifier(n_estimators=100)

In [44]:
for train_x, test_x in kfold.split(data_train['Stop_Content'], data_train['Sentiment']):
  train_X, test_X = data_train['Stop_Content'][train_x], data_train['Stop_Content'][test_x]
  train_y, test_y = data_train['Sentiment'][train_x], data_train['Sentiment'][test_x]
  temp_train_x, temp_test_x = transform_train_data(train_X, test_X, tfidf_vect)
  temp_train_y, temp_test_y = transform_test_data(train_y, test_y)

  clf.fit(temp_train_x, temp_train_y)
  pred_y = clf.predict(temp_test_x)

  pred_accuracy_score.append(accuracy_score(temp_test_y, pred_y))
  pred_precison_score.append(precision_score(temp_test_y, pred_y))
  pred_recall_score.append(recall_score(temp_test_y, pred_y))
  pred_f1_score.append(f1_score(temp_test_y, pred_y))

In [45]:
print("Train Data\n")
print("Accuracy Score: ", pred_accuracy_score)
print("Precision Score: ", pred_precison_score)
print("Recall Score: ", pred_recall_score)
print("F1 Score: ", pred_f1_score)

Train Data

Accuracy Score:  [0.7167630057803468, 0.7232658959537572, 0.7102601156069365, 0.7189306358381503, 0.7059248554913294, 0.7333815028901735, 0.7203757225433526, 0.7138728323699421, 0.7232658959537572, 0.7319364161849711]
Precision Score:  [0.726027397260274, 0.7299864314789688, 0.7302725968436155, 0.7253044654939107, 0.7215189873417721, 0.7461645746164575, 0.7445255474452555, 0.7335243553008596, 0.7263017356475301, 0.7387755102040816]
Recall Score:  [0.7340720221606648, 0.7451523545706371, 0.7049861495844876, 0.7423822714681441, 0.7105263157894737, 0.7409972299168975, 0.7063711911357341, 0.7091412742382271, 0.7534626038781164, 0.7520775623268698]
F1 Score:  [0.7300275482093662, 0.7374914324880055, 0.7174066243833687, 0.7337440109514032, 0.715980460572226, 0.7435719249478805, 0.7249466950959489, 0.7211267605633802, 0.7396329027872195, 0.7453671928620453]


In [46]:
pred_y=clf.predict(test_data_x)
print("Test Data\n")
print("Accuracy Score: ", accuracy_score(test_data_y, pred_y))
print("Precision Score: ", precision_score(test_data_y, pred_y))
print("Recall Score: ", recall_score(test_data_y, pred_y))
print("F1 Score: ", f1_score(test_data_y, pred_y))

Test Data

Accuracy Score:  0.7380560131795717
Precision Score:  0.7394678492239468
Recall Score:  0.7337733773377337
F1 Score:  0.736609607951408


### XG Boost

In [47]:
pred_accuracy_score=[]
pred_precison_score=[]
pred_recall_score=[]
pred_f1_score=[]

In [48]:
clf = XGBClassifier()

In [49]:
for train_x, test_x in kfold.split(data_train['Stop_Content'], data_train['Sentiment']):
  train_X, test_X = data_train['Stop_Content'][train_x], data_train['Stop_Content'][test_x]
  train_y, test_y = data_train['Sentiment'][train_x], data_train['Sentiment'][test_x]
  temp_train_x, temp_test_x = transform_train_data(train_X, test_X, tfidf_vect)
  temp_train_y, temp_test_y = transform_test_data(train_y, test_y)

  clf.fit(temp_train_x, temp_train_y)
  pred_y = clf.predict(temp_test_x)

  pred_accuracy_score.append(accuracy_score(temp_test_y, pred_y))
  pred_precison_score.append(precision_score(temp_test_y, pred_y))
  pred_recall_score.append(recall_score(temp_test_y, pred_y))
  pred_f1_score.append(f1_score(temp_test_y, pred_y))

In [50]:
print("Train Data\n")
print("Accuracy Score: ", pred_accuracy_score)
print("Precision Score: ", pred_precison_score)
print("Recall Score: ", pred_recall_score)
print("F1 Score: ", pred_f1_score)

Train Data

Accuracy Score:  [0.6423410404624278, 0.6488439306358381, 0.6221098265895953, 0.6401734104046243, 0.6228323699421965, 0.6488439306358381, 0.6416184971098265, 0.6452312138728323, 0.6401734104046243, 0.6329479768786127]
Precision Score:  [0.6118226600985222, 0.6187122736418511, 0.6016343207354443, 0.6108910891089109, 0.59765625, 0.6159135559921415, 0.6094961240310077, 0.6214511041009464, 0.6098039215686275, 0.6044921875]
Recall Score:  [0.8601108033240997, 0.8518005540166205, 0.8157894736842105, 0.8545706371191135, 0.8476454293628809, 0.868421052631579, 0.871191135734072, 0.8185595567867036, 0.8614958448753463, 0.8573407202216067]
F1 Score:  [0.7150259067357512, 0.7167832167832168, 0.6925338036449148, 0.7124711316397229, 0.7010309278350517, 0.7206896551724139, 0.7172177879133409, 0.7065152420800958, 0.7141216991963262, 0.709049255441008]


In [51]:
pred_y=clf.predict(test_data_x)
print("Test Data\n")
print("Accuracy Score: ", accuracy_score(test_data_y, pred_y))
print("Precision Score: ", precision_score(test_data_y, pred_y))
print("Recall Score: ", recall_score(test_data_y, pred_y))
print("F1 Score: ", f1_score(test_data_y, pred_y))

Test Data

Accuracy Score:  0.6414058209774849
Precision Score:  0.5980091883614089
Recall Score:  0.8591859185918592
F1 Score:  0.7051918735891649
